# What is a Mesh?

In PyVista, a mesh is any spatially referenced information and usually consists of geometrical representations of a surface or volume in 3D space. We commonly refer to any spatially referenced dataset as a mesh, so often the distinction between a mesh, a grid, and a volume can get fuzzy – but that does not matter in PyVista. If you have a dataset that is a surface mesh with 2D geometries like triangles, we call it a mesh, and if you have a dataset with 3D geometries like voxels, tetrahedra, hexahedra, etc., then we also call that a mesh! Why? Because it is simple that way.

In all spatially referenced datasets, there lies an underlying mesh structure – the connectivity or geometry between nodes to define cells. Whether those cells are 2D or 3D is not always of importance and we’ve worked hard to make PyVista work for datasets of either or mixed geometries so that you as a user do not have to get bogged down in the nuances.

## What is a point?

Points are the vertices of the mesh – the Cartesian coordinates of the underlying structure. All PyVista datasets (meshes!) have points and sometimes, you can have a mesh that only has points – like a point cloud.

For example, you can create a point cloud mesh using the [pyvista.PolyData](https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyData.html#pyvista.PolyData) class which is built for meshes that have 1D and 2D cell types (we’ll get into what a cell is briefly).

Let’s start with a point cloud – this is a mesh type that only has vertices. You can create one by defining a 2D array of Cartesian coordinates like so:

In [ ]:
import numpy as np
import pyvista as pv

pv.set_plot_theme("document")

points = np.random.rand(100, 3)
mesh = pv.PolyData(points)
mesh.plot(point_size=10, style="points")


But it’s important to note that most meshes have some sort of connectivity between points such as this gridded mesh:

In [ ]:
from pyvista import examples

mesh = examples.load_hexbeam()
cpos = [(6.20, 3.00, 7.50), (0.16, 0.13, 2.65), (-0.28, 0.94, -0.21)]

pl = pv.Plotter()
pl.add_mesh(mesh, show_edges=True, color="white")
pl.add_points(mesh.points, color="red", point_size=20)
pl.camera_position = cpos
pl.show()


Or this triangulated surface:

In [ ]:
mesh = examples.download_bunny_coarse()

pl = pv.Plotter()
pl.add_mesh(mesh, show_edges=True, color="white")
pl.add_points(mesh.points, color="red", point_size=2)
pl.camera_position = [(0.02, 0.30, 0.73), (0.02, 0.03, -0.022), (-0.03, 0.94, -0.34)]
pl.show()


### Exercise: Plotting points on one part of a sphere

Let's plot the sphere and further plot the points on the positive Z-axis.

In [ ]:
# Your code here


In [ ]:
import numpy as np
import pyvista as pv

mesh = pv.Sphere()
points = mesh.points
pl = pv.Plotter()
pl.add_mesh(mesh, show_edges=True, color="white")
pl.add_points(points[points[:, 2] > 0.0], color="red", point_size=10)
pl.show()


## What is a Cell?

A cell is the geometry between points that defines the connectivity or topology of a mesh. In the examples above, cells are defined by the lines (edges colored in black) connecting points (colored in red). For example, a cell in the beam example is a voxel defined by the region between eight points in that mesh:

In [ ]:
mesh = examples.load_hexbeam()

pl = pv.Plotter()
pl.add_mesh(mesh, show_edges=True, color="white")
pl.add_points(mesh.points, color="red", point_size=20)

single_cell = mesh.extract_cells(mesh.n_cells - 1)
pl.add_mesh(single_cell, color="pink", edge_color="blue", line_width=5, show_edges=True)

pl.camera_position = [(6.20, 3.00, 7.50), (0.16, 0.13, 2.65), (-0.28, 0.94, -0.21)]
pl.show()


Cells aren’t limited to voxels, they could be a triangle between three points, a line between two points, or even a single point could be its own cell (but that’s a special case).

### Exercise

Let's highlight the cell connected to the point with coordinates (0.5, 0.5, 0.5) in the beam above.

In [ ]:
# Your code here


## What are attributes?

Attributes are data values that live on either the points or cells of a mesh. In PyVista, we work with both point data and cell data and allow easy access to data dictionaries to hold arrays for attributes that live either on all points or on all cells of a mesh. These attributes can be accessed in a dictionary-like attribute attached to any PyVista mesh accessible as one of the following:

- [point_data](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.point_data.html#pyvista.DataSet.point_data)
- [cell_data](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.cell_data.html#pyvista.DataSet.cell_data)
- [field_data](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.field_data.html#pyvista.DataSet.field_data)

## Point Data

Point data refers to arrays of values (scalars, vectors, etc.) that live on each point of the mesh. Each element in an attribute array corresponds to a point in the mesh. Let’s create some point data for the beam mesh. When plotting, the values between points are interpolated across the cells.

In [ ]:
mesh.point_data["my point values"] = np.arange(mesh.n_points)
mesh.plot(scalars="my point values", cpos=cpos, show_edges=True)


## Cell Data

Cell data refers to arrays of values (scalars, vectors, etc.) that live throughout each cell of the mesh. That is the entire cell (2D face or 3D volume) is assigned the value of that attribute.

mesh.cell_data['my cell values'] = np.arange(mesh.n_cells)
mesh.plot(scalars='my cell values', cpos=cpos, show_edges=True)

Here’s a comparison of point data versus cell data and how point data is interpolated across cells when mapping colors. This is unlike cell data which has a single value across the cell’s domain:

In [ ]:
import pyvista as pv
from pyvista import examples

uni = examples.load_uniform()

pl = pv.Plotter(shape=(1, 2), border=False)
pl.add_mesh(uni, scalars="Spatial Point Data", show_edges=True)
pl.subplot(0, 1)
pl.add_mesh(uni, scalars="Spatial Cell Data", show_edges=True)
pl.show()


## Field Data

Field data is not directly associated with either the points or cells but still should be attached to the mesh. This may be a string array storing notes, or even indices of a Collision.

## Assigning Scalars to a Mesh

Here’s how we assign values to cell attributes and plot it. Here, we generate cube containing 6 faces and assign each face an integer from `range(6)` and then have it plotted.

Note how this varies from assigning scalars to each point

In [ ]:
cube = pv.Cube()
cube.cell_data["myscalars"] = range(6)

other_cube = cube.copy()
other_cube.point_data["myscalars"] = range(8)

pl = pv.Plotter(shape=(1, 2), border_width=1)
pl.add_mesh(cube, cmap="coolwarm")
pl.subplot(0, 1)
pl.add_mesh(other_cube, cmap="coolwarm")
pl.show()


We use [pyvista.PolyDataFilters.clean()](https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyDataFilters.clean.html#pyvista.PolyDataFilters.clean) to merge the faces of the cube since, by default, the cube is created with unmerged faces and duplicate points.

### Exercise

Set the height data to the points of the Stanford bunny mesh and plot the contours.

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Creating an Explicit Structured Grid {#ref_create_explicit_structured_grid}
====================================

Create an explicit structured grid from NumPy arrays.

Note this feature is only available for `vtk>=9`.


In [ ]:
import numpy as np

import pyvista as pv

ni, nj, nk = 4, 5, 6
si, sj, sk = 20, 10, 1

xcorn = np.arange(0, (ni + 1) * si, si)
xcorn = np.repeat(xcorn, 2)
xcorn = xcorn[1:-1]
xcorn = np.tile(xcorn, 4 * nj * nk)

ycorn = np.arange(0, (nj + 1) * sj, sj)
ycorn = np.repeat(ycorn, 2)
ycorn = ycorn[1:-1]
ycorn = np.tile(ycorn, (2 * ni, 2 * nk))
ycorn = np.transpose(ycorn)
ycorn = ycorn.flatten()

zcorn = np.arange(0, (nk + 1) * sk, sk)
zcorn = np.repeat(zcorn, 2)
zcorn = zcorn[1:-1]
zcorn = np.repeat(zcorn, (4 * ni * nj))

corners = np.stack((xcorn, ycorn, zcorn))
corners = corners.transpose()

if pv._vtk.VTK9:
    dims = np.asarray((ni, nj, nk)) + 1
    grid = pv.ExplicitStructuredGrid(dims, corners)
    grid = grid.compute_connectivity()
    grid.plot(show_edges=True)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Create a Kochanek Spline {#create_kochanek_spline_example}
========================

Create a Kochanek spline/polyline from a numpy array of XYZ vertices.


In [ ]:
import numpy as np

import pyvista as pv

Create a dataset to plot


In [ ]:
def make_points():
    """Helper to make XYZ points"""
    theta = np.linspace(-4 * np.pi, 4 * np.pi, 6)
    z = np.linspace(-2, 2, 6)
    r = z**2 + 1
    x = r * np.sin(theta)
    y = r * np.cos(theta)
    return np.column_stack((x, y, z))


points = make_points()
points[0:5, :]

Interpolate those points onto a parametric Kochanek spline


In [ ]:
# Create Kochanek spline with 6 interpolation points
p = pv.Plotter(shape=(3, 5))

c = [-1.0, -0.5, 0.0, 0.5, 1.0]
for i in range(5):
    kochanek_spline = pv.KochanekSpline(points, continuity=[c[i], c[i], c[i]], n_points=1000)
    p.subplot(0, i)
    p.add_text("c = " + str(c[i]))
    p.add_mesh(kochanek_spline, color="k", point_size=10)
    p.add_mesh(
        pv.PolyData(points),
        color="k",
        point_size=10,
        render_points_as_spheres=True,
    )

t = [-1.0, -0.5, 0.0, 0.5, 1.0]
for i in range(5):
    kochanek_spline = pv.KochanekSpline(points, tension=[t[i], t[i], t[i]], n_points=1000)
    p.subplot(1, i)
    p.add_text("t = " + str(t[i]))
    p.add_mesh(kochanek_spline, color="k")
    p.add_mesh(
        pv.PolyData(points),
        color="k",
        point_size=10,
        render_points_as_spheres=True,
    )

b = [-1.0, -0.5, 0.0, 0.5, 1.0]
for i in range(5):
    kochanek_spline = pv.KochanekSpline(points, bias=[b[i], b[i], b[i]], n_points=1000)
    p.subplot(2, i)
    p.add_text("b = " + str(b[i]))
    p.add_mesh(kochanek_spline, color="k")
    p.add_mesh(
        pv.PolyData(points),
        color="k",
        point_size=10,
        render_points_as_spheres=True,
    )

p.show(cpos="xy")

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Parametric Geometric Objects {#ref_parametric_example}
============================

Creating parametric objects


In [ ]:
from math import pi

import pyvista as pv

This example demonstrates how to plot parametric objects using pyvista

Supertoroid
===========


In [ ]:
supertoroid = pv.ParametricSuperToroid(n1=0.5)
supertoroid.plot(color="tan", smooth_shading=True)

Parametric Ellipsoid
====================


In [ ]:
# Ellipsoid with a long x axis
ellipsoid = pv.ParametricEllipsoid(10, 5, 5)
ellipsoid.plot(color="tan")

Partial Parametric Ellipsoid
============================


In [ ]:
# cool plotting direction
cpos = [
    (21.9930, 21.1810, -30.3780),
    (-1.1640, -1.3098, -0.1061),
    (0.8498, -0.2515, 0.4631),
]


# half ellipsoid
part_ellipsoid = pv.ParametricEllipsoid(10, 5, 5, max_v=pi / 2)
part_ellipsoid.plot(color="tan", smooth_shading=True, cpos=cpos)

Pseudosphere
============


In [ ]:
pseudosphere = pv.ParametricPseudosphere()
pseudosphere.plot(color="tan", smooth_shading=True)

Bohemian Dome
=============


In [ ]:
bohemiandome = pv.ParametricBohemianDome()
bohemiandome.plot(color="tan")

Bour
====


In [ ]:
bour = pv.ParametricBour()
bour.plot(color="tan")

Boy\'s Surface
==============


In [ ]:
boy = pv.ParametricBoy()
boy.plot(color="tan")

Catalan Minimal
===============


In [ ]:
catalanminimal = pv.ParametricCatalanMinimal()
catalanminimal.plot(color="tan")

Conic Spiral
============


In [ ]:
conicspiral = pv.ParametricConicSpiral()
conicspiral.plot(color="tan")

Cross Cap
=========


In [ ]:
crosscap = pv.ParametricCrossCap()
crosscap.plot(color="tan")

Dini
====


In [ ]:
dini = pv.ParametricDini()
dini.plot(color="tan")

Enneper
=======


In [ ]:
enneper = pv.ParametricEnneper()
enneper.plot(cpos="yz")

Figure-8 Klein
==============


In [ ]:
figure8klein = pv.ParametricFigure8Klein()
figure8klein.plot()

Henneberg
=========


In [ ]:
henneberg = pv.ParametricHenneberg()
henneberg.plot(color="tan")

Klein
=====


In [ ]:
klein = pv.ParametricKlein()
klein.plot(color="tan")

Kuen
====


In [ ]:
kuen = pv.ParametricKuen()
kuen.plot(color="tan")

Mobius
======


In [ ]:
mobius = pv.ParametricMobius()
mobius.plot(color="tan")

Plucker Conoid
==============


In [ ]:
pluckerconoid = pv.ParametricPluckerConoid()
pluckerconoid.plot(color="tan")

Random Hills
============


In [ ]:
randomhills = pv.ParametricRandomHills()
randomhills.plot(color="tan")

Roman
=====


In [ ]:
roman = pv.ParametricRoman()
roman.plot(color="tan")

Super Ellipsoid
===============


In [ ]:
superellipsoid = pv.ParametricSuperEllipsoid(n1=0.1, n2=2)
superellipsoid.plot(color="tan")

Torus
=====


In [ ]:
torus = pv.ParametricTorus()
torus.plot(color="tan")

Circular Arc
============


In [ ]:
pointa = [-1, 0, 0]
pointb = [0, 1, 0]
center = [0, 0, 0]
resolution = 100

arc = pv.CircularArc(pointa, pointb, center, resolution)

pl = pv.Plotter()
pl.add_mesh(arc, color='k', line_width=4)
pl.show_bounds()
pl.view_xy()
pl.show()

Extruded Half Arc
=================


In [ ]:
pointa = [-1, 0, 0]
pointb = [1, 0, 0]
center = [0, 0, 0]
resolution = 100

arc = pv.CircularArc(pointa, pointb, center, resolution)
poly = arc.extrude([0, 0, 1])
poly.plot(color="tan", cpos='iso', show_edges=True)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Pixel Art of ALIEN MONSTERS {#pixel_art_example}
===========================

Here we use `pyvista.Box`{.interpreted-text role="func"} to make [pixel
art](https://en.wikipedia.org/wiki/Pixel_art). Pixel string
[source](https://commons.wikimedia.org/wiki/File:Noto_Emoji_Pie_1f47e.svg)
and [license](https://github.com/googlefonts/noto-emoji#license).


In [ ]:
import pyvista as pv
from pyvista.demos import logo

Convert pixel art to an array
=============================


In [ ]:
alien_str = """
    %         %
      %     %
    % % % % % %
  % %   % %   % %
% % % % % % % % % %
%   % % % % % %   %
%   %         %   %
%   % %     % %   %
      %     %
    %         %
"""


alien = []
for line in alien_str.splitlines()[1:]:  # skip first linebreak
    if not line:
        continue
    if len(line) < 20:
        line += (20 - len(line)) * ' '
    alien.append([line[i : i + 2] == '% ' for i in range(0, len(line), 2)])

Define function to draw pixels
==============================

Define a helper function to add pixel boxes to plotter.


In [ ]:
def draw_pixels(plotter, pixels, center, color):
    bounds = [
        center[0] - 1.0,
        center[0] + 1.0,
        center[1] - 1.0,
        center[1] + 1.0,
        -10.0,
        +10.0,
    ]
    for rows in pixels:
        for pixel in rows:
            if pixel:
                box = pv.Box(bounds=bounds)
                plotter.add_mesh(box, color=color)
            bounds[0] += 2.0
            bounds[1] += 2.0
        bounds[0] = center[0] - 1.0
        bounds[1] = center[0] + 1.0
        bounds[2] += -2.0
        bounds[3] += -2.0
    return plotter

Now you can plot a pixel art of ALIEN MONSTERS.


In [ ]:
# Display MONSTERS
p = pv.Plotter()
p = draw_pixels(p, alien, [-22.0, 22.0], "green")
p = draw_pixels(p, alien, [0.0, 22.0], "green")
p = draw_pixels(p, alien, [22.0, 22.0], "green")
p = draw_pixels(p, alien, [-22.0, 0.0], "blue")
p = draw_pixels(p, alien, [0.0, 0.0], "blue")
p = draw_pixels(p, alien, [22.0, 0.0], "blue")
p = draw_pixels(p, alien, [-22.0, -22.0], "red")
p = draw_pixels(p, alien, [0.0, -22.0], "red")
p = draw_pixels(p, alien, [22.0, -22.0], "red")

text = logo.text_3d("ALIEN MONSTERS", depth=10.0)
text.points *= 4.0
text.translate([-20.0, 24.0, 0.0], inplace=True)

p.add_mesh(text, color="yellow")
p.show(cpos="xy")

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Platonic Solids {#platonic_example}
===============

PyVista wraps the `vtk.vtkPlatonicSolidSource` filter as
`pyvista.PlatonicSolid`{.interpreted-text role="func"}.


In [ ]:
import numpy as np

import pyvista as pv
from pyvista import examples

We can either use the generic
`PlatonicSolid() <pyvista.PlatonicSolid>`{.interpreted-text role="func"}
and specify the different kinds of solids to generate, or we can use the
thin wrappers:

> -   `pyvista.Tetrahedron`{.interpreted-text role="func"}
> -   `pyvista.Octahedron`{.interpreted-text role="func"}
> -   `pyvista.Dodecahedron`{.interpreted-text role="func"}
> -   `pyvista.Icosahedron`{.interpreted-text role="func"}
> -   `pyvista.Cube`{.interpreted-text role="func"} (implemented via a
>     different filter)

Let\'s generate all the Platonic solids, along with the `teapotahedron
<pyvista.examples.downloads.download_teapot>`{.interpreted-text
role="func"}.


In [ ]:
kinds = [
    'tetrahedron',
    'cube',
    'octahedron',
    'dodecahedron',
    'icosahedron',
]
centers = [
    (0, 1, 0),
    (0, 0, 0),
    (0, 2, 0),
    (-1, 0, 0),
    (-1, 2, 0),
]

solids = [pv.PlatonicSolid(kind, radius=0.4, center=center) for kind, center in zip(kinds, centers)]

# download and align teapotahedron
teapot = examples.download_teapot()
teapot.rotate_x(90, inplace=True)
teapot.rotate_z(-45, inplace=True)
teapot.scale(0.16, inplace=True)
teapot.points += np.array([-1, 1, 0]) - teapot.center
solids.append(teapot)

Now let\'s plot them all.

::: {.note}
::: {.title}
Note
:::

VTK has known issues when rendering shadows on certain window sizes. Be
prepared to experiment with the `window_size` parameter. An initial
window size of `(1000, 1000)` seems to work well, which can be manually
resized without issue.
:::


In [ ]:
p = pv.Plotter(window_size=[1000, 1000])
for ind, solid in enumerate(solids):
    # only use smooth shading for the teapot
    smooth_shading = ind == len(solids) - 1
    p.add_mesh(
        solid, color='silver', smooth_shading=smooth_shading, specular=1.0, specular_power=10
    )
p.view_vector((5.0, 2, 3))
p.add_floor('-z', lighting=True, color='tan', pad=1.0)
p.enable_shadows()
p.show()

The Platonic solids come with cell scalars that index each face of the
solids.


In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Create Point Cloud {#create_point_cloud}
==================

Create a `pyvista.PolyData`{.interpreted-text role="class"} object from
a point cloud of vertices and scalar arrays for those points.


In [ ]:
import numpy as np

import pyvista as pv
from pyvista import examples

Point clouds are generally constructed in the
`pyvista.PolyData`{.interpreted-text role="class"} class and can easily
have scalar/vector data arrays associated with the point cloud. In this
example, we\'ll work a bit backwards using a point cloud that that is
available from our `examples` module. This however is no different than
creating a PyVista mesh with your own NumPy arrays of vertice locations.


In [ ]:
# Define some helpers - ignore these and use your own data!
def generate_points(subset=0.02):
    """A helper to make a 3D NumPy array of points (n_points by 3)"""
    dataset = examples.download_lidar()
    ids = np.random.randint(low=0, high=dataset.n_points - 1, size=int(dataset.n_points * subset))
    return dataset.points[ids]


points = generate_points()
# Print first 5 rows to prove its a numpy array (n_points by 3)
# Columns are (X Y Z)
points[0:5, :]

Now that you have a NumPy array of points/vertices either from our
sample data or your own project, creating a PyVista mesh of those points
is simply:


In [ ]:
point_cloud = pv.PolyData(points)
point_cloud

And we can even do a sanity check


In [ ]:
np.allclose(points, point_cloud.points)

And now that we have a PyVista mesh, we can plot it. Note that we add an
option to use eye dome lighting - this is a shading technique to improve
depth perception with point clouds (learn more in
`ref_edl`{.interpreted-text role="ref"}).


In [ ]:
point_cloud.plot(eye_dome_lighting=True)

Now what if you have data attributes (scalar/vector arrays) that you\'d
like to associate with every node of your mesh? You can easily add NumPy
data arrays that have a length equal to the number of points in the mesh
along the first axis. For example, lets add a few arrays to this new
`point_cloud` mesh.

Make an array of scalar values with the same length as the points array.
Each element in this array will correspond to points at the same index:


In [ ]:
# Make data array using z-component of points array
data = points[:, -1]

# Add that data to the mesh with the name "uniform dist"
point_cloud["elevation"] = data

And now we can plot the point cloud with that random data. PyVista is
smart enough to plot the scalar array you added by default. Note that
this time, we specify to render every point as its own sphere.


In [ ]:
point_cloud.plot(render_points_as_spheres=True)

That data is kind of boring, right? You can also add data arrays with
more than one scalar value - perhaps a vector with three elements?
Let\'s make a little function that will compute vectors for every node
in the point cloud and add those vectors to the mesh.

This time, we\'re going to create a totally new, random point cloud.


In [ ]:
# Create random XYZ points
points = np.random.rand(100, 3)
# Make PolyData
point_cloud = pv.PolyData(points)


def compute_vectors(mesh):
    origin = mesh.center
    vectors = mesh.points - origin
    vectors = vectors / np.linalg.norm(vectors, axis=1)[:, None]
    return vectors


vectors = compute_vectors(point_cloud)
vectors[0:5, :]

In [ ]:
point_cloud['vectors'] = vectors

Now we can make arrows using those vectors using the glyph filter (see
`glyph_example`{.interpreted-text role="ref"} for more details).


In [ ]:
arrows = point_cloud.glyph(
    orient='vectors',
    scale=False,
    factor=0.15,
)

# Display the arrows
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color='maroon', point_size=10.0, render_points_as_spheres=True)
plotter.add_mesh(arrows, color='lightblue')
# plotter.add_point_labels([point_cloud.center,], ['Center',],
#                          point_color='yellow', point_size=20)
plotter.show_grid()
plotter.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Create a PointSet {#create_pointset_example}
=================

A `pyvista.PointSet`{.interpreted-text role="class"} is a concrete class
representing a set of points that specifies the interface for datasets
that explicitly use \"point\" arrays to represent geometry. This class
is useful for improving the performance of filters on point clouds.

This example shows the performance improvement when clipping using the
`pyvista.DataSet.clip`{.interpreted-text role="func"} filter on a
`pyvista.PointSet`{.interpreted-text role="class"}.


In [ ]:
import time

import pyvista as pv
from pyvista import examples

lidar = examples.download_lidar()

tstart = time.time()
clipped = lidar.clip(origin=(0, 0, 1.76e3), normal=(0, 0, 1))
t_elapsed = time.time() - tstart
print(f"Time to clip with a PolyData {t_elapsed:.2f} seconds.")

Plot the clipped polydata


In [ ]:
clipped.plot(show_scalar_bar=False)

Show the performance improvement when using a PointSet. This is only
available with VTK \>= 9.1.0.


In [ ]:
# pset = lidar.cast_to_pointset(deep=False)

if pv.vtk_version_info >= (9, 1):
    lidar_pset = lidar.cast_to_pointset()
    tstart = time.time()
    clipped_pset = lidar_pset.clip(origin=(0, 0, 1.76e3), normal=(0, 0, 1))
    t_elapsed = time.time() - tstart
    print(f"Time to clip with a PointSet {t_elapsed:.2f} seconds.")

Plot the same dataset.

::: {.note}
::: {.title}
Note
:::

PyVista must still create an intermediate PolyData to be able to plot,
so there is no performance improvement when using a
`pyvista.PointSet`{.interpreted-text role="class"}
:::


In [ ]:
if pv.vtk_version_info >= (9, 1):
    clipped_pset.plot(show_scalar_bar=False)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Create PolyData {#ref_create_poly}
===============

Creating a PolyData (triangulated surface) object from NumPy arrays of
the vertices and faces.


In [ ]:
import numpy as np

import pyvista as pv

A PolyData object can be created quickly from numpy arrays. The vertex
array contains the locations of the points in the mesh and the face
array contains the number of points of each face and the indices of the
vertices which comprise that face.


In [ ]:
# mesh points
vertices = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0], [0.5, 0.5, -1]])

# mesh faces
faces = np.hstack([[4, 0, 1, 2, 3], [3, 0, 1, 4], [3, 1, 2, 4]])  # [square, triangle, triangle]

surf = pv.PolyData(vertices, faces)

# plot each face with a different color
surf.plot(scalars=np.arange(3), cpos=[-1, 1, 0.5])

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Creating a Spline {#ref_create_spline}
=================

Create a spline/polyline from a numpy array of XYZ vertices


In [ ]:
import numpy as np

import pyvista as pv

Create a dataset to plot


In [ ]:
def make_points():
    """Helper to make XYZ points"""
    theta = np.linspace(-4 * np.pi, 4 * np.pi, 100)
    z = np.linspace(-2, 2, 100)
    r = z**2 + 1
    x = r * np.sin(theta)
    y = r * np.cos(theta)
    return np.column_stack((x, y, z))


points = make_points()
points[0:5, :]

Now let\'s make a function that can create line cells on a
`pyvista.PolyData`{.interpreted-text role="class"} mesh given that the
points are in order for the segments they make.


In [ ]:
def lines_from_points(points):
    """Given an array of points, make a line set"""
    poly = pv.PolyData()
    poly.points = points
    cells = np.full((len(points) - 1, 3), 2, dtype=np.int_)
    cells[:, 1] = np.arange(0, len(points) - 1, dtype=np.int_)
    cells[:, 2] = np.arange(1, len(points), dtype=np.int_)
    poly.lines = cells
    return poly


line = lines_from_points(points)
line

In [ ]:
line["scalars"] = np.arange(line.n_points)
tube = line.tube(radius=0.1)
tube.plot(smooth_shading=True)

That tube has sharp edges at each line segment. This can be mitigated by
creating a single PolyLine cell for all of the points


In [ ]:
def polyline_from_points(points):
    poly = pv.PolyData()
    poly.points = points
    the_cell = np.arange(0, len(points), dtype=np.int_)
    the_cell = np.insert(the_cell, 0, len(points))
    poly.lines = the_cell
    return poly


polyline = polyline_from_points(points)
polyline["scalars"] = np.arange(polyline.n_points)
tube = polyline.tube(radius=0.1)
tube.plot(smooth_shading=True)

You could also interpolate those points onto a parametric spline


In [ ]:
# Create spline with 1000 interpolation points
spline = pv.Spline(points, 1000)

Plot spline as a tube


In [ ]:
# add scalars to spline and plot it
spline["scalars"] = np.arange(spline.n_points)
tube = spline.tube(radius=0.1)
tube.plot(smooth_shading=True)

The spline can also be plotted as a plain line


In [ ]:
# generate same spline with 400 interpolation points
spline = pv.Spline(points, 400)

# plot without scalars
spline.plot(line_width=4, color="k")

Ribbons
=======

Ayy of the lines from the examples above can be used to create ribbons.
Take a look at the `pyvista.PolyDataFilters.ribbon`{.interpreted-text
role="func"} filter.


In [ ]:
ribbon = spline.compute_arc_length().ribbon(width=0.75, scalars='arc_length')
ribbon.plot(color=True)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Creating a Structured Surface {#ref_create_structured}
=============================

Create a StructuredGrid surface from NumPy arrays


In [ ]:
import numpy as np

import pyvista as pv
from pyvista import examples

From NumPy Meshgrid
===================

Create a simple meshgrid using NumPy


In [ ]:
# Make data
x = np.arange(-10, 10, 0.25)
y = np.arange(-10, 10, 0.25)
x, y = np.meshgrid(x, y)
r = np.sqrt(x**2 + y**2)
z = np.sin(r)

Now pass the NumPy meshgrid to PyVista


In [ ]:
# Create and plot structured grid
grid = pv.StructuredGrid(x, y, z)
grid.plot()

In [ ]:
# Plot mean curvature as well
grid.plot_curvature(clim=[-1, 1])

Generating a structured grid is a one-liner in this module, and the
points from the resulting surface can be accessed as a NumPy array:


In [ ]:
grid.points

From XYZ Points
===============

Quite often, you might be given a set of coordinates (XYZ points) in a
simple tabular format where there exists some structure such that grid
could be built between the nodes you have. A great example is found in
[pyvista-support\#16](https://github.com/pyvista/pyvista-support/issues/16)
where a structured grid that is rotated from the cartesian reference
frame is given as just XYZ points. In these cases, all that is needed to
recover the grid is the dimensions of the grid ([nx]{.title-ref} by
[ny]{.title-ref} by [nz]{.title-ref}) and that the coordinates are
ordered appropriately.

For this example, we will create a small dataset and rotate the
coordinates such that they are not on orthogonal to cartesian reference
frame.


In [ ]:
def make_point_set():
    """Ignore the contents of this function. Just know that it returns an
    n by 3 numpy array of structured coordinates."""
    n, m = 29, 32
    x = np.linspace(-200, 200, num=n) + np.random.uniform(-5, 5, size=n)
    y = np.linspace(-200, 200, num=m) + np.random.uniform(-5, 5, size=m)
    xx, yy = np.meshgrid(x, y)
    A, b = 100, 100
    zz = A * np.exp(-0.5 * ((xx / b) ** 2.0 + (yy / b) ** 2.0))
    points = np.c_[xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)]
    foo = pv.PolyData(points)
    foo.rotate_z(36.6, inplace=True)
    return foo.points


# Get the points as a 2D NumPy array (N by 3)
points = make_point_set()
points[0:5, :]

Now pretend that the (n by 3) NumPy array above are coordinates that you
have, possibly from a file with three columns of XYZ points.

We simply need to recover the dimensions of the grid that these points
make and then we can generate a
`pyvista.StructuredGrid`{.interpreted-text role="class"} mesh.

Let\'s preview the points to see what we are dealing with:


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.scatter(points[:, 0], points[:, 1], c=points[:, 2])
plt.axis("image")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()

In the figure above, we can see some inherit structure to the points and
thus we could connect the points as a structured grid. All we need to
know are the dimensions of the grid present. In this case, we know
(because we made this dataset) the dimensions are `[29, 32, 1]`, but you
might not know the dimensions of your pointset. There are a few ways to
figure out the dimensionality of structured grid including:

-   manually counting the nodes along the edges of the pointset
-   using a technique like principle component analysis to strip the
    rotation from the dataset and count the unique values along each
    axis for the new;y projected dataset.


In [ ]:
# Once you've figured out your grid's dimensions, simple create the
# :class:`pyvista.StructuredGrid` as follows:

mesh = pv.StructuredGrid()
# Set the coordinates from the numpy array
mesh.points = points
# set the dimensions
mesh.dimensions = [29, 32, 1]

# and then inspect it!
mesh.plot(show_edges=True, show_grid=True, cpos="xy")

Extending a 2D StructuredGrid to 3D
===================================

A 2D `pyvista.StructuredGrid`{.interpreted-text role="class"} mesh can
be extended into a 3D mesh. This is highly applicable when wanting to
create a terrain following mesh in earth science research applications.

For example, we could have a `pyvista.StructuredGrid`{.interpreted-text
role="class"} of a topography surface and extend that surface to a few
different levels and connect each \"level\" to create the 3D terrain
following mesh.

Let\'s start with a simple example by extending the wave mesh to 3D


In [ ]:
struct = examples.load_structured()
struct.plot(show_edges=True)

In [ ]:
top = struct.points.copy()
bottom = struct.points.copy()
bottom[:, -1] = -10.0  # Wherever you want the plane

vol = pv.StructuredGrid()
vol.points = np.vstack((top, bottom))
vol.dimensions = [*struct.dimensions[0:2], 2]
vol.plot(show_edges=True)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Create Triangulated Surface {#triangulated_surface}
===========================

Create a surface from a set of points through a Delaunay triangulation.


In [ ]:
import numpy as np

import pyvista as pv

Simple Triangulations
=====================

First, create some points for the surface.


In [ ]:
# Define a simple Gaussian surface
n = 20
x = np.linspace(-200, 200, num=n) + np.random.uniform(-5, 5, size=n)
y = np.linspace(-200, 200, num=n) + np.random.uniform(-5, 5, size=n)
xx, yy = np.meshgrid(x, y)
A, b = 100, 100
zz = A * np.exp(-0.5 * ((xx / b) ** 2.0 + (yy / b) ** 2.0))

# Get the points as a 2D NumPy array (N by 3)
points = np.c_[xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)]
points[0:5, :]

Now use those points to create a point cloud PyVista data object. This
will be encompassed in a `pyvista.PolyData`{.interpreted-text
role="class"} object.


In [ ]:
# simply pass the numpy points to the PolyData constructor
cloud = pv.PolyData(points)
cloud.plot(point_size=15)

Now that we have a PyVista data structure of the points, we can perform
a triangulation to turn those boring discrete points into a connected
surface.


In [ ]:
surf = cloud.delaunay_2d()
surf.plot(show_edges=True)

Masked Triangulations
=====================


In [ ]:
x = np.arange(10, dtype=float)
xx, yy, zz = np.meshgrid(x, x, [0])
points = np.column_stack((xx.ravel(order="F"), yy.ravel(order="F"), zz.ravel(order="F")))
# Perturb the points
points[:, 0] += np.random.rand(len(points)) * 0.3
points[:, 1] += np.random.rand(len(points)) * 0.3
# Create the point cloud mesh to triangulate from the coordinates
cloud = pv.PolyData(points)
cloud

Run the triangulation on these points


In [ ]:
surf = cloud.delaunay_2d()
surf.plot(cpos="xy", show_edges=True)

Note that some of the outer edges are unconstrained and the
triangulation added unwanted triangles. We can mitigate that with the
`alpha` parameter.


In [ ]:
surf = cloud.delaunay_2d(alpha=1.0)
surf.plot(cpos="xy", show_edges=True)

We could also add a polygon to ignore during the triangulation via the
`edge_source` parameter.


In [ ]:
# Define a polygonal hole with a clockwise polygon
ids = [22, 23, 24, 25, 35, 45, 44, 43, 42, 32]

# Create a polydata to store the boundary
polygon = pv.PolyData()
# Make sure it has the same points as the mesh being triangulated
polygon.points = points
# But only has faces in regions to ignore
polygon.faces = np.insert(ids, 0, len(ids))

surf = cloud.delaunay_2d(alpha=1.0, edge_source=polygon)

p = pv.Plotter()
p.add_mesh(surf, show_edges=True)
p.add_mesh(polygon, color="red", opacity=0.5)
p.show(cpos="xy")

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Plot Truss-like FEA Solution with Cylinders {#create_truss}
===========================================

Plot connections between points in 3D as cylinders, colored by scalars.


In [ ]:
import numpy as np

import pyvista

Define the points and elements of the truss. Call them `nodes` here as
it comes from finite element analysis.


In [ ]:
nodes = [
    [0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [4.0, 3.0, 0.0],
    [4.0, 0.0, 0.0],
    [0.0, 1.0, 2.0],
    [4.0, 1.0, 2.0],
    [4.0, 3.0, 2.0],
]


edges = np.array(
    [
        [0, 4],
        [1, 4],
        [3, 4],
        [5, 4],
        [6, 4],
        [3, 5],
        [2, 5],
        [5, 6],
        [2, 6],
    ]
)

# We must "pad" the edges to indicate to vtk how many points per edge
padding = np.empty(edges.shape[0], int) * 2
padding[:] = 2
edges_w_padding = np.vstack((padding, edges.T)).T
edges_w_padding

Plot the truss while rendering the lines as tubes.


In [ ]:
mesh = pyvista.PolyData(nodes, edges_w_padding)

colors = range(edges.shape[0])
mesh.plot(
    scalars=colors,
    render_lines_as_tubes=True,
    style='wireframe',
    line_width=10,
    cmap='jet',
    show_scalar_bar=False,
    background='w',
)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Creating a Uniform Grid
=======================

Create a simple uniform grid from a 3D NumPy array of values.


In [ ]:
import numpy as np

import pyvista as pv

Take a 3D NumPy array of data values that holds some spatial data where
each axis corresponds to the XYZ cartesian axes. This example will
create a `pyvista.UniformGrid`{.interpreted-text role="class"} object
that will hold the spatial reference for a 3D grid which a 3D NumPy
array of values can be plotted against.


Create the 3D NumPy array of spatially referenced data. This is
spatially referenced such that the grid is 20 by 5 by 10 (nx by ny by
nz)


In [ ]:
values = np.linspace(0, 10, 1000).reshape((20, 5, 10))
values.shape

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
grid.dimensions = np.array(values.shape) + 1

# Edit the spatial reference
grid.origin = (100, 33, 55.6)  # The bottom left corner of the data set
grid.spacing = (1, 5, 2)  # These are the cell sizes along each axis

# Add the data values to the cell data
grid.cell_data["values"] = values.flatten(order="F")  # Flatten the array!

# Now plot the grid!
grid.plot(show_edges=True)

Don\'t like cell data? You could also add the NumPy array to the point
data of a `pyvista.UniformGrid`{.interpreted-text role="class"}. Take
note of the subtle difference when setting the grid dimensions upon
initialization.


In [ ]:
# Create the 3D NumPy array of spatially referenced data
# This is spatially referenced such that the grid is 20 by 5 by 10
#   (nx by ny by nz)
values = np.linspace(0, 10, 1000).reshape((20, 5, 10))
values.shape

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values on the
#   POINT data
grid.dimensions = values.shape

# Edit the spatial reference
grid.origin = (100, 33, 55.6)  # The bottom left corner of the data set
grid.spacing = (1, 5, 2)  # These are the cell sizes along each axis

# Add the data values to the cell data
grid.point_data["values"] = values.flatten(order="F")  # Flatten the array!

# Now plot the grid!
grid.plot(show_edges=True)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Creating an Unstructured Grid {#ref_create_unstructured}
=============================

Create an irregular, unstructured grid from NumPy arrays.


In [ ]:
import numpy as np
import vtk

import pyvista as pv

An unstructured grid can be created directly from NumPy arrays. This is
useful when creating a grid from scratch or copying it from another
format. See
[vtkUnstructuredGrid](https://www.vtk.org/doc/nightly/html/classvtkUnstructuredGrid.html)
for available cell types and their descriptions.


In [ ]:
# offset array.  Identifies the start of each cell in the cells array
offset = np.array([0, 9])

# Contains information on the points composing each cell.
# Each cell begins with the number of points in the cell and then the points
# composing the cell
cells = np.array([8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 15])

# cell type array. Contains the cell type of each cell
cell_type = np.array([vtk.VTK_HEXAHEDRON, vtk.VTK_HEXAHEDRON])

# in this example, each cell uses separate points
cell1 = np.array(
    [
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 1],
        [0, 1, 1],
    ]
)

cell2 = np.array(
    [
        [0, 0, 2],
        [1, 0, 2],
        [1, 1, 2],
        [0, 1, 2],
        [0, 0, 3],
        [1, 0, 3],
        [1, 1, 3],
        [0, 1, 3],
    ]
)

# points of the cell array
points = np.vstack((cell1, cell2))

# create the unstructured grid directly from the numpy arrays
# The offset is optional and will be either calculated if not given (VTK version < 9),
# or is not necessary anymore (VTK version >= 9)
grid = pv.UnstructuredGrid(offset, cells, cell_type, points)

# For cells of fixed sizes (like the mentioned Hexahedra), it is also possible to use the
# simplified dictionary interface. This automatically calculates the cell array with types
# and offsets. Note that for mixing with additional cell types, just the appropriate key needs to be
# added to the dictionary.
cells_hex = np.arange(16).reshape([2, 8])
# = np.array([[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15]])
grid = pv.UnstructuredGrid({vtk.VTK_HEXAHEDRON: cells_hex}, points)

# plot the grid (and suppress the camera position output)
_ = grid.plot(show_edges=True)

UnstructuredGrid with Shared Points
===================================

The next example again creates an unstructured grid containing
hexahedral cells, but using common points between the cells.


In [ ]:
# these points will all be shared between the cells
points = np.array(
    [
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.5, 0.0, 0.0],
        [1.0, 1.0, 0.0],
        [1.0, 0.5, 0.0],
        [0.0, 1.0, 0.0],
        [0.5, 1.0, 0.0],
        [0.0, 0.5, 0.0],
        [0.5, 0.5, 0.0],
        [1.0, 0.0, 0.5],
        [1.0, 0.0, 1.0],
        [0.0, 0.0, 0.5],
        [0.0, 0.0, 1.0],
        [0.5, 0.0, 0.5],
        [0.5, 0.0, 1.0],
        [1.0, 1.0, 0.5],
        [1.0, 1.0, 1.0],
        [1.0, 0.5, 0.5],
        [1.0, 0.5, 1.0],
        [0.0, 1.0, 0.5],
        [0.0, 1.0, 1.0],
        [0.5, 1.0, 0.5],
        [0.5, 1.0, 1.0],
        [0.0, 0.5, 0.5],
        [0.0, 0.5, 1.0],
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 1.0],
    ]
)


# Each cell in the cell array needs to include the size of the cell
# and the points belonging to the cell.  In this example, there are 8
# hexahedral cells that have common points between them.
cells = np.array(
    [
        [8, 0, 2, 8, 7, 11, 13, 25, 23],
        [8, 2, 1, 4, 8, 13, 9, 17, 25],
        [8, 7, 8, 6, 5, 23, 25, 21, 19],
        [8, 8, 4, 3, 6, 25, 17, 15, 21],
        [8, 11, 13, 25, 23, 12, 14, 26, 24],
        [8, 13, 9, 17, 25, 14, 10, 18, 26],
        [8, 23, 25, 21, 19, 24, 26, 22, 20],
        [8, 25, 17, 15, 21, 26, 18, 16, 22],
    ]
).ravel()

# each cell is a VTK_HEXAHEDRON
celltypes = np.empty(8, dtype=np.uint8)
celltypes[:] = vtk.VTK_HEXAHEDRON

# the offset array points to the start of each cell (via flat indexing)
offset = np.array([0, 9, 18, 27, 36, 45, 54, 63])

# Effectively, when visualizing a VTK unstructured grid, it will
# sequentially access the cell array by first looking at each index of
# cell array (based on the offset array), and then read the number of
# points based on the first value of the cell.  In this case, the
# VTK_HEXAHEDRON is described by 8 points.

# for example, the 5th cell would be accessed by vtk with:
start_of_cell = offset[4]
n_points_in_cell = cells[start_of_cell]
indices_in_cell = cells[start_of_cell + 1 : start_of_cell + n_points_in_cell + 1]
print(indices_in_cell)

Finally, create the unstructured grid and plot it


In [ ]:
# if you are using VTK 9.0 or newer, you do not need to input the offset array:
# grid = pv.UnstructuredGrid(cells, celltypes, points)

# if you are not using VTK 9.0 or newer, you must use the offset array
grid = pv.UnstructuredGrid(offset, cells, celltypes, points)

# Alternate versions:
grid = pv.UnstructuredGrid({vtk.VTK_HEXAHEDRON: cells.reshape([-1, 9])[:, 1:]}, points)
grid = pv.UnstructuredGrid(
    {vtk.VTK_HEXAHEDRON: np.delete(cells, np.arange(0, cells.size, 9))}, points
)

# plot the grid (and suppress the camera position output)
_ = grid.plot(show_edges=True)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Plotting Glyphs (Vectors or PolyData) {#glyph_example}
=====================================

Use vectors in a dataset to plot and orient glyphs/geometric objects.


In [ ]:
import numpy as np

import pyvista as pv
from pyvista import examples

Glyphying can be done via the
`pyvista.DataSetFilters.glyph`{.interpreted-text role="func"} filter


In [ ]:
mesh = examples.download_carotid().threshold(145, scalars="scalars")
mask = mesh['scalars'] < 210
mesh['scalars'][mask] = 0  # null out smaller vectors

# Make a geometric object to use as the glyph
geom = pv.Arrow()  # This could be any dataset

# Perform the glyph
glyphs = mesh.glyph(orient="vectors", scale="scalars", factor=0.003, geom=geom)

# plot using the plotting class
pl = pv.Plotter()
pl.add_mesh(glyphs, show_scalar_bar=False, lighting=False, cmap='coolwarm')
pl.camera_position = [
    (146.53, 91.28, 21.70),
    (125.00, 94.45, 19.81),
    (-0.086, 0.007, 0.996),
]  # view only part of the vector field
cpos = pl.show(return_cpos=True)

Another approach is to load the vectors directly to the mesh object and
then access the `pyvista.DataSet.arrows`{.interpreted-text role="attr"}
property.


In [ ]:
sphere = pv.Sphere(radius=3.14)

# make cool swirly pattern
vectors = np.vstack(
    (
        np.sin(sphere.points[:, 0]),
        np.cos(sphere.points[:, 1]),
        np.cos(sphere.points[:, 2]),
    )
).T

# add and scale
sphere["vectors"] = vectors * 0.3
sphere.set_active_vectors("vectors")

# plot just the arrows
sphere.arrows.plot()

Plot the arrows and the sphere.


In [ ]:
p = pv.Plotter()
p.add_mesh(sphere.arrows, lighting=False, scalar_bar_args={'title': "Vector Magnitude"})
p.add_mesh(sphere, color="grey", ambient=0.6, opacity=0.5, show_edges=False)
p.show()

Subset of Glyphs
================

Sometimes you might not want glyphs for every node in the input dataset.
In this case, you can choose to build glyphs for a subset of the input
dataset by using a merging tolerance. Here we specify a merging
tolerance of five percent which equates to five percent of the bounding
box\'s length.


In [ ]:
# Example dataset with normals
mesh = examples.load_random_hills()

# create a subset of arrows using the glyph filter
arrows = mesh.glyph(scale="Normals", orient="Normals", tolerance=0.05)

p = pv.Plotter()
p.add_mesh(arrows, color="black")
p.add_mesh(mesh, scalars="Elevation", cmap="terrain", smooth_shading=True)
p.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Table of Glyphs {#glyph_table_example}
===============

`vtk` supports tables of glyphs from which glyphs are looked up. This
example demonstrates this functionality.


In [ ]:
import numpy as np

import pyvista as pv

We can allow tables of glyphs in a backward-compatible way by allowing a
sequence of geometries as well as single (scalar) geometries to be
passed as the `geom` kwarg of
`pyvista.DataSetFilters.glyph`{.interpreted-text role="func"}. An
`indices` optional keyword specifies the index of each glyph geometry in
the table, and it has to be the same length as `geom` if specified. If
it is absent a default value of `range(len(geom))` is assumed.


In [ ]:
# get dataset for the glyphs: supertoroids in xy plane
# use N random kinds of toroids over a mesh with 27 points
N = 5
values = np.arange(N)  # values for scalars to look up glyphs by


# taken from:
# rng = np.random.default_rng()
# params = rng.uniform(0.5, 2, size=(N, 2))  # (n1, n2) parameters for the toroids
params = np.array(
    [
        [1.56821334, 0.99649769],
        [1.08247844, 1.83758874],
        [1.49598881, 0.83495047],
        [1.52442129, 0.89600688],
        [1.92212387, 0.78096621],
    ]
)

geoms = [pv.ParametricSuperToroid(n1=n1, n2=n2) for n1, n2 in params]

# get dataset where to put glyphs
x, y, z = np.mgrid[:3.0, :3.0, :3.0]
mesh = pv.StructuredGrid(x, y, z)

# add random scalars
# rng_int = rng.integers(0, N, size=x.size)
rng_int = np.array(
    [4, 1, 2, 0, 4, 0, 1, 4, 3, 1, 1, 3, 3, 4, 3, 4, 4, 3, 3, 2, 2, 1, 1, 1, 2, 0, 3]
)
mesh.point_data['scalars'] = rng_int

# construct the glyphs on top of the mesh; don't scale by scalars now
glyphs = mesh.glyph(geom=geoms, indices=values, scale=False, factor=0.3, rng=(0, N - 1))

# create plotter and add our glyphs with some nontrivial lighting
plotter = pv.Plotter()
plotter.add_mesh(glyphs, specular=1, specular_power=15, smooth_shading=True, show_scalar_bar=False)
plotter.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Rotations {#rotate_example}
=========

Rotations of a mesh about its axes. In this model, the x axis is from
the left to right; the y axis is from bottom to top; and the z axis
emerges from the image. The camera location is the same in all four
images.


In [ ]:
import pyvista as pv
from pyvista import examples

Define camera and axes
======================

Define camera and axes. Setting axes origin to `(3.0, 3.0, 3.0)`.


In [ ]:
mesh = examples.download_cow()
mesh.points /= 1.5  # scale the mesh

camera = pv.Camera()
camera.position = (30.0, 30.0, 30.0)
camera.focal_point = (5.0, 5.0, 5.0)

axes = pv.Axes(show_actor=True, actor_scale=2.0, line_width=5)
axes.origin = (3.0, 3.0, 3.0)

Original Mesh
=============

Plot original mesh. Add axes actor to Plotter.


In [ ]:
p = pv.Plotter()

p.add_text("Mesh", font_size=24)
p.add_actor(axes.actor)
p.camera = camera
p.add_mesh(mesh)

p.show()

Rotation about the x axis
=========================

Plot the mesh rotated about the x axis every 60 degrees. Add the axes
actor to the Plotter and set the axes origin to the point of rotation.


In [ ]:
p = pv.Plotter()

p.add_text("X-Axis Rotation", font_size=24)
p.add_actor(axes.actor)
p.camera = camera

for i in range(6):
    rot = mesh.rotate_x(60 * i, point=axes.origin, inplace=False)
    p.add_mesh(rot)

p.show()

Rotation about the y axis
=========================

Plot the mesh rotated about the y axis every 60 degrees. Add the axes
actor to the Plotter and set the axes origin to the point of rotation.


In [ ]:
p = pv.Plotter()

p.add_text("Y-Axis Rotation", font_size=24)
p.camera = camera
p.add_actor(axes.actor)

for i in range(6):
    rot = mesh.rotate_y(60 * i, point=axes.origin, inplace=False)
    p.add_mesh(rot)

p.show()

Rotation about the z axis
=========================

Plot the mesh rotated about the z axis every 60 degrees. Add axes actor
to the Plotter and set the axes origin to the point of rotation.


In [ ]:
p = pv.Plotter()

p.add_text("Z-Axis Rotation", font_size=24)
p.camera = camera
p.add_actor(axes.actor)

for i in range(6):
    rot = mesh.rotate_z(60 * i, point=axes.origin, inplace=False)
    p.add_mesh(rot)

p.show()

Rotation about a custom vector
==============================

Plot the mesh rotated about a custom vector every 60 degrees. Add the
axes actor to the Plotter and set axes origin to the point of rotation.


In [ ]:
p = pv.Plotter()

p.add_text("Custom Vector Rotation", font_size=24)
p.camera = camera
p.add_actor(axes.actor)
for i in range(6):
    rot = mesh.copy()
    rot.rotate_vector(vector=(1, 1, 1), angle=60 * i, point=axes.origin)
    p.add_mesh(rot)

p.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Sample Function: Perlin Noise in 2D
===================================

Here we use `pyvista.core.imaging.sample_function`{.interpreted-text
role="func"} to sample Perlin noise over a region to generate random
terrain.

Perlin noise is atype of gradient noise often used by visual effects
artists to increase the appearance of realism in computer graphics.
Source: <https://en.wikipedia.org/wiki/Perlin_noise>

The development of Perlin Noise has allowed computer graphics artists to
better represent the complexity of natural phenomena in visual effects
for the motion picture industry.


In [ ]:
import pyvista as pv

Generate Perlin Noise over a StructuredGrid
===========================================

Feel free to change the values of `freq` to change the shape of the
\"mountains\". For example, lowering the frequency will make the terrain
seem more like hills rather than mountains.


In [ ]:
freq = [0.689, 0.562, 0.683]
noise = pv.perlin_noise(1, freq, (0, 0, 0))
sampled = pv.sample_function(noise, bounds=(-10, 10, -10, 10, -10, 10), dim=(500, 500, 1))

Warp by scalar
==============

Here we warp by scalar to give the terrain some height based on the
value of the Perlin noise. This is necessary to the terrain its shape.


In [ ]:
mesh = sampled.warp_by_scalar('scalars')
mesh = mesh.extract_surface()

# clean and smooth a little to reduce Perlin noise artifacts
mesh = mesh.smooth(n_iter=100, inplace=False, relaxation_factor=1)

# This makes the "water" level look flat.
z = mesh.points[:, 2]
diff = z.max() - z.min()

# water level at 70%  (change this to change the water level)
water_percent = 0.7
water_level = z.max() - water_percent * diff
mesh.points[z < water_level, 2] = water_level

Show the terrain as a contour plot


In [ ]:
# make the water blue
rng = z.max() - z.min()
clim = (z.max() - rng * 1.65, z.max())

pl = pv.Plotter()
pl.add_mesh(
    mesh,
    scalars=z,
    cmap='gist_earth',
    n_colors=10,
    show_scalar_bar=False,
    smooth_shading=True,
    clim=clim,
)
pl.show()

Show the terrain with custom lighting and shadows


In [ ]:
pl = pv.Plotter(lighting=None)
pl.add_light(pv.Light((3, 1, 0.5), show_actor=True, positional=True, cone_angle=90, intensity=1.2))
pl.add_mesh(mesh, cmap='gist_earth', show_scalar_bar=False, smooth_shading=True, clim=clim)
pl.enable_shadows = True
pl.show()

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Sample Function: Perlin Noise in 3D
===================================

Here we use `pyvista.core.imaging.sample_function`{.interpreted-text
role="func"} to sample Perlin noise over a region to generate random
terrain.

Video games like Minecraft use Perlin noise to create terrain. Here, we
create a voxelized mesh similar to a Minecraft \"cave\".


In [ ]:
import pyvista as pv

Generate Perlin Noise over a 3D StructuredGrid
==============================================

Feel free to change the values of `freq` to change the shape of the
\"caves\". For example, lowering the frequency will make the caves
larger and more expansive, while a higher frequency in any direction
will make the caves appear more \"vein-like\" and less open.

Change the threshold to reduce or increase the percent of the terrain
that is open or closed


In [ ]:
freq = (1, 1, 1)
noise = pv.perlin_noise(1, freq, (0, 0, 0))
grid = pv.sample_function(noise, [0, 3.0, -0, 1.0, 0, 1.0], dim=(120, 40, 40))
out = grid.threshold(0.02)
out

color limits without blue


In [ ]:
mn, mx = [out['scalars'].min(), out['scalars'].max()]
clim = (mn, mx * 1.8)

out.plot(
    cmap='gist_earth_r',
    background='white',
    show_scalar_bar=False,
    lighting=True,
    clim=clim,
    show_edges=False,
)

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme('document')

Terrain Following Mesh {#terrain_following_mesh_example}
======================

Use a topographic surface to create a 3D terrain-following mesh.

Terrain following meshes are common in the environmental sciences, for
instance in hydrological modelling (see [Maxwell
2013](https://www.sciencedirect.com/science/article/abs/pii/S0309170812002564)
and [ParFlow](https://parflow.org)).

In this example, we demonstrate a simple way to make a 3D grid/mesh that
follows a given topographic surface. In this example, it is important to
note that the given digital elevation model (DEM) is structured (gridded
and not triangulated): this is common for DEMs.


In [ ]:
import numpy as np

import pyvista as pv
from pyvista import examples

Download a gridded topography surface (DEM)


In [ ]:
dem = examples.download_crater_topo()
dem

Now let\'s subsample and extract an area of interest to make this
example simple (also the DEM we just load is pretty big). Since the DEM
we loaded is a `pyvista.UniformGrid`{.interpreted-text role="class"}
mesh, we can use the
`pyvista.UniformGridFilters.extract_subset`{.interpreted-text
role="func"} filter:


In [ ]:
subset = dem.extract_subset((500, 900, 400, 800, 0, 0), (5, 5, 1))
subset.plot(cpos="xy")

Now that we have a region of interest for our terrain following mesh,
lets make a 3D surface of that DEM:


In [ ]:
terrain = subset.warp_by_scalar()
terrain

In [ ]:
terrain.plot()

And now we have a 3D structured surface of the terrain! We can now
extend that structured surface into a 3D mesh to form a terrain
following grid. To do this, we first our cell spacings in the
z-direction (these start from the terrain surface). Then we repeat the
XYZ structured coordinates of the terrain mesh and decrease each Z level
by our Z cell spacing. Once we have those structured coordinates, we can
create a `pyvista.StructuredGrid`{.interpreted-text role="class"}.


In [ ]:
z_cells = np.array([25] * 5 + [35] * 3 + [50] * 2 + [75, 100])

xx = np.repeat(terrain.x, len(z_cells), axis=-1)
yy = np.repeat(terrain.y, len(z_cells), axis=-1)
zz = np.repeat(terrain.z, len(z_cells), axis=-1) - np.cumsum(z_cells).reshape((1, 1, -1))

mesh = pv.StructuredGrid(xx, yy, zz)
mesh["Elevation"] = zz.ravel(order="F")
mesh

In [ ]:
cpos = [
    (1826736.796308761, 5655837.275274233, 4676.8405505181745),
    (1821066.1790519988, 5649248.765538796, 943.0995128226014),
    (-0.2797856225380979, -0.27966946337594883, 0.9184252809434081),
]

mesh.plot(show_edges=True, lighting=False, cpos=cpos)